# Coursework 

**Initialize packages**

In [ ]:
using JuMP, HiGHS, Plots, DataFrames, CSV, Statistics, StatsBase, Printf, Colors  ;

**Reading Data**

In [ ]:
flex_avail = DataFrame(CSV.File("input_files/Flexible_load_availability.csv"));

In [ ]:
generators = DataFrame(CSV.File("input_files/Generators_data.csv"));


In [ ]:
new_col_names = string.(generators.R_ID);  # Convert to String

In [ ]:
gen_var = DataFrame(CSV.File("input_files/Generators_variability.csv"))
gen_var=gen_var[:,2:35];
#gen_var=gen_var[:,2:end];
rename!(gen_var, new_col_names);

In [ ]:
load = DataFrame(CSV.File("input_files/Load_data.csv"));
load=load[:,9:15];

In [ ]:
network = DataFrame(CSV.File("input_files/Network.csv"));

### Problem formulation


**SETS**

$\begin{align}
& N & \text{Set for zones [1:6]}\\[6pt]
& L & \text{Set for Transmission Lines [1:10]}\\[6pt]
& H  &  \text{Set of hours in the year [1:8736]}\\[6pt]
& H_2 & \text{Set of hours for Storage Operation [2:8736]}\\[6pt]
& Storage & \text{Set for storage[6 batteries, one per zone]}\\[6pt]
& Gen & \text{Set for generators }\\[6pt]
& Id_{STORAGE} & \text{Set for storage Id}\\[6pt]
& Id_{Gen} & \text{Set for Gen Id }\\[6pt]
\end{align}$

**Parameters**
- $Investment_g$ annuitized capacity of Generator $g$ investment cost (annual cost, $/MW-year)
- $Investment_s$ annuitized capacity of Storage $s$ investment cost (annual cost, $/MW-year)
- $FixedOM_g$, the fixed operations and maintenance cost of Generator $g$(\$/MW-year)
- $FixedOM_s$, the fixed operations and maintenance cost of Storage $s$(\$/MW-year)
- $VarOM_g$, the variable operations and maintenance cost (\$/MWh);
- $HeatRate_g$, the rate of fuel consumed per unit of electricity produced (MMBtu/MWh);
- $FuelCost_g$, the cost of fuel (\$/MMBtu)
- $NSECost$, the opportunity cost or penalty incurred for involuntary non-served energy (\$/MWh); and
- $Demand_h$, is the demand in each hour (MW).
- $MaxFlow_{ij}$, the maximum allowable flow along the line from $i$ to $j$ (MW)
- $CF_{g,h}$, hourly profile of Generator $g$ (per unit).
- $Eff_s$, efficiency of the charge/discharge cycle.
- $SOC^{max}$, maximum state of char capacity (MWh).

**Variables**
$\begin{align}
&\text{Capacity variable only for new build:}&\\[6pt]
& CAP_{g} \geq 0 & \forall \quad g \in Generators\\[6pt]
&\text{Capacity variable for Storage:}&\\[6pt]
& 0 \leq CAP_s \leq 100 MW & \forall \quad s \in Storage\\[6pt]
&\text{Generation variable:}&\\[6pt]
& GEN_{g,h} \geq 0 & \forall \quad g \in NEW_{GEN}  ,  & \forall \quad h \in Hours \\[6pt]
&\text{Not Served Energy:}&\\[6pt]
& NSE_{h} \geq 0 & \forall \quad h \in H \\[6pt]
&\text{Flow variable:}&\\[6pt]
& FLOW_{l,h} & \forall \quad l \in Transmission_{Lines}  ,  & \forall \quad h \in Hours\\[6pt]
&\text{Discharge variable:}&\\[6pt]
& DISCHARGE_{s,h} \geq 0 & \forall \quad s \in Storage  ,  & \forall \quad h \in Hours\\[6pt]
&\text{Charge variable:}&\\[6pt]
& CHARGE_{s,h} \geq 0 & \forall \quad s \in Storage  ,  & \forall \quad h \in Hours\\[6pt]
&\text{State of Charge variable:}&\\[6pt]
& 0\% \leq SOC_{s,h} \leq 100\%  & \forall \quad s \in Storage  ,  & \forall \quad h \in Hours\\[6pt]
\end{align}$

**Objective Function**
$\begin{align}
\min &\sum_{g \in Gen} FixedCost_g \times CAP_g + \sum_{g\in Gen}\sum_{h \in H} VarCost_g \times GEN_{g,h} +\\[6pt]
&\quad\quad \sum_{h \in H} NSECost \times NSE_h  + \sum_{s \in Storage} FixedCost_s \times CAP_s \\[6pt]
\end{align}$

$\begin{align}
\ FixedCost_g = Investment_g + FixedOM_g\\[6pt]
\ VarCost_g = VarOM_g + FuelCost_g \times HeatRate_g \\[6pt]
\ FixedCost_s = Investment_s + FixedOM_s\\[6pt]
\end{align}$

**Constraints**
$\begin{align}
&\text{Capacity constraint for nuclear:}&\\[6pt]
& GEN_{Nuclear,h} = CAP_{Nuclear} & \forall h \in H\\[6pt]
&\text{Capacity constraint for new generators:}&\\[6pt]
& 0 \leq  GEN_{g,h} \leq CAP_g \times CF_{g,h} & \forall \quad g \in GEN, h \in H\\[6pt]
&\text{Supply demand balance constraint:}&\\[6pt]
&\sum_{g \in Gen} GEN_{g,h,i} - Demand_{h,i} + \sum_{s \in Storage} (DISCHARGE_{s,h,i} -CHARGE_{s,h,i} ) + NSE_{h,i}  = \sum_{j \in J_i} FLOW_{ij} & \forall \quad i \in \text{N} &, & \forall \quad h \in \text{H}\\
&\text{Thermal limit constraint:}&\\[6pt]
& |FLOW_{ij}| \leq MaxFlow_{ij} & \forall \quad i \in \text{N} &, & \forall j \in J_i \\
&\text{Flow direction constraint:}&\\[6pt]
& FLOW_{ij} = - FLOW_{ji} & \forall \quad i, j \in \text{N} \\
&\text{State of Charge constraint:}&\\[6pt]
& SOC_{s,h}=SOC_{s,h-1} + \big(CHARGE_{s,h} * Eff_s - \frac{DISCHARGE_{s,h}}{Eff_s}\big)  & \forall \quad s \in Storage, h \in H_2 \\[6pt]
&\text{Initial and Final State of Charge constraint:}&\\[6pt]
& SOC_{s,1} = SOC_{s,t_{end}} = 50\% *SOC_{s}^{max}& \forall \quad s \in Storage\\[6pt]
&\text{Disharge Rate constraint:}&\\[6pt]
& 0 \leq DISCHARGE_{s,h} \leq CAP_s  & \forall \quad s \in Storage,  h \in H\\[6pt]
&\text{Charge Rate constraint:}&\\[6pt]
& 0 \leq CHARGE_{s,h} \leq  CAP_s  & \forall \quad s \in Storage,  h \in H\\[6pt]
&\text{State of Charge constraint:}&\\[6pt]
& 0 \leq SOC_{s,h} \leq SOC_{s}^{max}& \forall \quad s \in Storage, h \in H\\[6pt]
\end{align}$


**CASE 1: NO FLEXIBILITY**

**Sets**

In [ ]:
N=[1,2,3,4,5,6]; #zones
L=[1,2,3,4,5,6,7,8,9,10]; # transmission lines

In [ ]:
# The set of hours in the demand DataFrame
H = load.Time_index; # same for every zone

In [ ]:
H_2 = H[2:end]; # to avoid the first hour 

In [ ]:
# Storage
storage = generators[generators.Resource .== "battery", :];

In [ ]:
# Generators
gen = generators[generators.Resource .!= "battery", :];

In [ ]:
# Storage id
Id_storage=storage.R_ID;
# Generators id
Id_gen=gen.R_ID;
#Zones
Zone_gen=gen.Zone;

**Define the Incident Matrix**

In [ ]:
# Define the Incident Matrix
A = [
    1   -1   0    0    0    0;
    1    0  -1    0    0    0;
    0    1  -1    0    0    0;
    0    1   0   -1    0    0;
    0    0   1   -1    0    0;
    0    0   0    1   -1    0;
    0    0   0    1    0   -1;
    1    0   0    0   -1    0;
    0    1   0    0   -1    0;
    0    0   1    0   -1    0
];

**Model**

In [ ]:
NoFlex_Model = Model(HiGHS.Optimizer) # using the HiGHS open source solver

**Variables**

In [ ]:
@variables(NoFlex_Model, begin
    # Generating capacity (MW)
    CAP[g in Id_gen] >= 0
    # Storage capacity (MW)
    100 >= CAP_stor[s in Id_storage] >= 0
    
    # Generation in each hour (MWh)
    GEN[g in Id_gen, h in H] >= 0
    
    # Non-served energy in each hour (MWh) – currently uses zones 1:6
    NSE[h in H, z in N] >= 0
    
    # Flow
    FLOW[l in L, h in H]
    
    # Storage operations (charge/discharge in MW, SOC in MWh)
    
    0 <= DISCHARGE[s in Id_storage, h in H]
    0 <= CHARGE[s in Id_storage, h in H]    
    0 <= SOC[s in Id_storage, h in H]       
    
end);





**Constraints**

In [ ]:
@constraints(NoFlex_Model, begin

#Inflexibility of Nuclear Plants
NuclearInflexibility[id in Id_gen ∩ gen[gen.Resource .== "nuclear", :R_ID], h in H], GEN[id, h] == CAP[id]

#Generation Capacity
Generation[id in Id_gen,h in H], GEN[id,h]<=CAP[id] * gen_var[h,string(id)]   

#Supply Demand Balance
cBalance1[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 1,:R_ID]) - load.Load_MW_z1[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==1,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==1,:R_ID]) + NSE[h,1]== sum(A[l,1]*FLOW[l,h] for l in L) ;  
cBalance2[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 2,:R_ID]) - load.Load_MW_z2[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==2,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==2,:R_ID]) + NSE[h,2]== sum(A[l,2]*FLOW[l,h] for l in L) ;   
cBalance3[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 3,:R_ID]) - load.Load_MW_z3[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==3,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==3,:R_ID]) + NSE[h,3]== sum(A[l,3]*FLOW[l,h] for l in L) ;
cBalance4[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 4,:R_ID]) - load.Load_MW_z4[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==4,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==4,:R_ID]) + NSE[h,4]== sum(A[l,4]*FLOW[l,h] for l in L) ;  
cBalance5[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 5,:R_ID]) - load.Load_MW_z5[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==5,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==5,:R_ID]) + NSE[h,5]== sum(A[l,5]*FLOW[l,h] for l in L) ;  
cBalance6[h in H], sum(GEN[id,h] for id in gen[gen.Zone .== 6,:R_ID]) - load.Load_MW_z6[h] +  sum(DISCHARGE[id_s,h] for id_s in storage[storage.Zone.==6,:R_ID])-sum(CHARGE[id_s,h] for id_s in storage[storage.Zone.==6,:R_ID]) + NSE[h,6]== sum(A[l,6]*FLOW[l,h] for l in L) ;

#Thermal constraint
Positive_max_flow[l in L, h in H], FLOW[l,h] <= network[l,:Line_Max_Flow_MW]
Negative_max_flow[l in L, h in H], FLOW[l,h] >= -network[l,:Line_Max_Flow_MW]

#State of charge
StateOfCharge[s in Id_storage, h in H_2], SOC[s,h] == SOC[s,h-1] + storage[storage.R_ID.==s,:Charge_efficiency][1]*CHARGE[s,h] - DISCHARGE[s,h]/storage[storage.R_ID.==s,:Discharge_efficiency][1] # State of charge of storage units

#States initial condition for SOC level (assumes half SOC)
StateOfcharge_ini[s in Id_storage], SOC[s,1].==0.5*CAP_stor[s].*storage.Max_Duration[storage.R_ID.==s] 

#States final condition for SOC level (assumes half SOC)
StateOfcharge_end[s in Id_storage], SOC[s,8736].==0.5*CAP_stor[s].*storage.Max_Duration[storage.R_ID.==s] 

# Storage operations (charge/discharge in MW, SOC in MWh)
limitdis[s in Id_storage, h in H], DISCHARGE[s,h] <= CAP_stor[s]
limitcha[s in Id_storage, h in H], CHARGE[s,h]<= CAP_stor[s]
limitSOC[s in Id_storage, h in H], SOC[s,h] .<= CAP_stor[s].*storage.Max_Duration[storage.R_ID.==s]

end);

**Objective function**

In [ ]:
@objective(NoFlex_Model,Min,
sum(
(sum(gen[gen.R_ID.==id, :Inv_cost_per_MWyr])+sum(gen[gen.R_ID.==id, :Fixed_OM_cost_per_MWyr]))*CAP[id]+
sum(sum(gen[gen.R_ID.==id,:Var_OM_cost_per_MWh] .+ sum(gen[gen.R_ID.==id,:Heat_rate_MMBTU_per_MWh].*gen[gen.R_ID.==id,:Cost_per_MMBtu]))
*GEN[id,h] for h in H) for id in Id_gen)+
sum(NSE[h, z] for h in H, z in N)*9000+
sum((sum(storage[storage.R_ID.==id_s,:Inv_cost_per_MWyr]).+sum(storage[storage.R_ID.==id_s,:Fixed_OM_cost_per_MWyr]))*CAP_stor[id_s] for id_s in Id_storage)
)


In [ ]:
optimize!(NoFlex_Model)

**Font**

In [ ]:
default(
    fontfamily = "Arial",  # Change this to your desired font
    titlefont = font(12),  # Title font size
    guidefont = font(12),  # Axis labels font size
    tickfont = font(12),   # Tick labels font size
    legendfont = font(8)  # Legend font size
)

In [ ]:
plot(load.Load_MW_z1+load.Load_MW_z2+load.Load_MW_z3+load.Load_MW_z4+load.Load_MW_z5+load.Load_MW_z6, xlims=(2550,2574) ,label="")
title!("Highest demand day")
xaxis!("Hour")
yaxis!("MWh")

In [ ]:

# Create empty arrays to store generator, hour, and generation values
generators = Int[]
hours = Int[]
generation_values = Float64[]

# Loop through each generator and hour, and store values
for g in Id_gen
    for h in H
        push!(generators, g)
        push!(hours, h)
        push!(generation_values, value(GEN[g, h]))
    end
end

# Create DataFrame from the collected values
df_gen_values = DataFrame(generators=generators, hour=hours, generation=generation_values)
# Merge df_gen_values with gen, renaming columns to align properly
df_gen_values = innerjoin(df_gen_values, gen, on=:generators => :R_ID)  # Merges on generators = R_ID

# Container to store each subplot
plot_list = []

# Loop through each zone and create its corresponding plot
for z in unique(df_gen_values.Zone)
    # Filter data for the current zone
    df_zone = filter(row -> row.Zone == z, df_gen_values)

    # Skip if no generators in this zone
    if nrow(df_zone) == 0
        println("No generators in zone $z")
        continue
    end

    # Create a new plot for this zone
    p = plot(
        title = "Generation Over Time (Zone $z)",
        xlabel = "Hour", ylabel = "Generation (MWh)",
        xlims = (0, 48)
    )

    # Plot each generator in this zone
    for g in unique(df_zone.generators)
        df_g = df_zone[df_zone.generators .== g, :]
        resource_name = df_g.Resource[1]
        plot!(p, df_g.hour, df_g.generation, label = "$resource_name", linewidth = 2)
        resource_name = df_g.Resource[1]
    end

    # Add the current plot to the list
    push!(plot_list, p)
end

# Combine all the plots into a subplot grid.
# Adjust layout, overall figure size, and spacing as needed.
final_plot = plot(plot_list..., layout = (3, 2), size = (1200, 800), subplot_padding = 10)
display(final_plot)



In [ ]:
# Extract storage capacities
stor_capacities = [(s, value(CAP_stor[s])) for s in Id_storage]

# Convert to a DataFrame
df_stor_capacities = DataFrame(storage_unit=first.(stor_capacities), 
                               capacity=last.(stor_capacities))

# Print the storage capacities
println(df_stor_capacities)

In [ ]:
# Storage unit ID (Change as needed)
s_id = 58;  

# Create empty vectors for SOC and hours
soc_values = Float64[]
hours = Int[]

# Loop through each hour to extract SOC values for storage unit 's_id'
for h in H
    push!(soc_values, value(SOC[s_id, h]))  # Store SOC values
    push!(hours, h)  # Store the corresponding hour
end

# Create a plot
plot(hours, soc_values, label="SOC", linewidth=2, linestyle=:solid, color=:blue, xlims=(8700, maximum(H)))

# Add labels and title
xlabel!("Hour")
ylabel!("SOC (MWh)")
title!("State of Charge Over Time for Storage Unit $s_id")


In [ ]:
# Extract nodal prices for each node over time
nodal_prices_node1 = Dict(h => dual(cBalance1[h, 1]) for h in H)  # Node 1
nodal_prices_node2 = Dict(h => dual(cBalance2[h, 1]) for h in H)  # Node 2
nodal_prices_node3 = Dict(h => dual(cBalance3[h, 1]) for h in H)  # Node 3
nodal_prices_node4 = Dict(h => dual(cBalance4[h, 1]) for h in H)  # Node 4
nodal_prices_node5 = Dict(h => dual(cBalance5[h, 1]) for h in H)  # Node 5
nodal_prices_node6 = Dict(h => dual(cBalance6[h, 1]) for h in H)  # Node 6
# Display sample prices for each node
println("Nodal price at node 1, hour 1: ", nodal_prices_node1[1])
println("Nodal price at node 2, hour 1: ", nodal_prices_node2[1])
println("Nodal price at node 3, hour 1: ", nodal_prices_node3[1])
println("Nodal price at node 4, hour 1: ", nodal_prices_node4[1])
println("Nodal price at node 5, hour 1: ", nodal_prices_node5[1])
println("Nodal price at node 6, hour 1: ", nodal_prices_node6[1])

# Convert nodal prices into vectors
nodal_prices_vector1 = [nodal_prices_node1[h] for h in H]
nodal_prices_vector2 = [nodal_prices_node2[h] for h in H]
nodal_prices_vector3 = [nodal_prices_node3[h] for h in H]
nodal_prices_vector4 = [nodal_prices_node4[h] for h in H]
nodal_prices_vector5 = [nodal_prices_node5[h] for h in H]
nodal_prices_vector6 = [nodal_prices_node6[h] for h in H]

# Create DataFrame (optional, if needed for further analysis)
nodal_prices_df = DataFrame(
    Hour = H,
    Node_1 = nodal_prices_vector1,
    Node_2 = nodal_prices_vector2,
    Node_3 = nodal_prices_vector3,
    Node_4 = nodal_prices_vector4,
    Node_5 = nodal_prices_vector5,
    Node_6 = nodal_prices_vector6
)


# Plot nodal prices for Node 1, Node 2, and Node 3
p12 = plot(H, nodal_prices_vector1, xlabel="Hour", ylabel="Nodal Price (USD/MWh)",
           title="Nodal Prices Over Time", label="Node 1", lw=2,
           xlims=(0, 48), ylims=(0, 400),legend=:topright)

plot!(H, nodal_prices_vector2, label="Node 2", lw=2)
plot!(H, nodal_prices_vector3, label="Node 3", lw=2)
plot!(H, nodal_prices_vector4, label="Node 4", lw=2)
plot!(H, nodal_prices_vector5, label="Node 5", lw=2)
plot!(H, nodal_prices_vector6, label="Node 6", lw=2)


display(p12)

In [ ]:
#Average price of electricicty
avg1=sum(nodal_prices_vector1)/length(nodal_prices_vector1);#zone1
avg2=sum(nodal_prices_vector2)/length(nodal_prices_vector2);#zone1
avg3=sum(nodal_prices_vector3)/length(nodal_prices_vector3);#zone1
avg4=sum(nodal_prices_vector4)/length(nodal_prices_vector4);#zone1
avg5=sum(nodal_prices_vector5)/length(nodal_prices_vector5);#zone1
avg6=sum(nodal_prices_vector6)/length(nodal_prices_vector6);#zone1
avg_total=(avg1+avg2+avg3+avg4+avg5+avg6)/6

println("Average price in Region 1 = $avg1")
println("Average price in Region 2 = $avg2")
println("Average price in Region 3 = $avg3")
println("Average price in Region 4 = $avg4")
println("Average price in Region 5 = $avg5")
println("Average price in Region 6 = $avg6")
println("Average price of all regions = $avg_total")


In [ ]:


# Define unique colors for each plot
colors = [:blue, :red, :green, :orange, :purple, :brown]

# Container for each individual plot
plot_list = []
a=0;
for i in N
    nse_values = Float64[]
    hours = Int[]
    # Loop through each hour to extract SOC values for storage unit 's_id'
    for h in H
        push!(nse_values, value(NSE[h, i]))  # Store SOC values
        push!(hours, h)                     # Store the corresponding hour
        a=a+value(NSE[h, i])
    end
    # Create the plot for this series with its own color and title
    p = plot(hours, nse_values, label="NSE$(i)", linewidth=2,
             linestyle=:solid, color=colors[i],
             xlims=(0, maximum(H)),
             xlabel="Hour", ylabel="NSE (MWh)",
             title="NSE Zone $(i)")
    push!(plot_list, p)
end

# Arrange the 6 plots in a grid layout (3 rows x 2 columns)
plot(plot_list..., layout=(2, 3), size=(700,500), subplot_padding=10)
#print(a)


In [ ]:

Total_Load=sum(load.Load_MW_z1+load.Load_MW_z2+load.Load_MW_z3+load.Load_MW_z4+load.Load_MW_z5+load.Load_MW_z6)

obj_val = objective_value(NoFlex_Model)
obj_val/Total_Load



In [ ]:
objective_value(NoFlex_Model)

**Highest Demand Day**

In [ ]:
plot(load.Load_MW_z1+load.Load_MW_z2+load.Load_MW_z3+load.Load_MW_z4+load.Load_MW_z5+load.Load_MW_z6, xlims=(2538,2586) ,label="")
title!("Highest demand day")
xaxis!("Hour")
yaxis!("MWh")






**Generators**

In [ ]:
# Generators
plot_list = []

# Loop through each zone and create its corresponding plot
for z in unique(df_gen_values.Zone)
    # Filter data for the current zone
    df_zone = filter(row -> row.Zone == z, df_gen_values)

    # Skip if no generators in this zone
    if nrow(df_zone) == 0
        println("No generators in zone $z")
        continue
    end

    # Create a new plot for this zone
    p = plot(
        title = "Generation Over Time (Zone $z)",
        xlabel = "Hour", ylabel = "Generation (MWh)",
        xlims = (2538,2586)
    )

    # Plot each generator in this zone
    for g in unique(df_zone.generators)
        df_g = df_zone[df_zone.generators .== g, :]
        resource_name = df_g.Resource[1]
        plot!(p, df_g.hour, df_g.generation, label = "$resource_name", linewidth = 2)
        resource_name = df_g.Resource[1]
    end

    # Add the current plot to the list
    push!(plot_list, p)
end

# Combine all the plots into a subplot grid.
# Adjust layout, overall figure size, and spacing as needed.
final_plot = plot(plot_list..., layout = (3, 2), size = (1200, 800), subplot_padding = 10)
display(final_plot)


**Nodal Prices**

In [ ]:
#Nodal prices
# Plot nodal prices for Node 1, Node 2, and Node 3
p12 = plot(H, nodal_prices_vector1, xlabel="Hour", ylabel="Nodal Price (USD/MWh)",
           title="Nodal Prices Over Time", label="Node 1", lw=2,
           xlims=(2538,2586), ylims=(0, 1000),legend=:topright)

plot!(H, nodal_prices_vector2, label="Node 2", lw=2)
plot!(H, nodal_prices_vector3, label="Node 3", lw=2)
plot!(H, nodal_prices_vector4, label="Node 4", lw=2)
plot!(H, nodal_prices_vector5, label="Node 5", lw=2)
plot!(H, nodal_prices_vector6, label="Node 6", lw=2)


display(p12)

Unserved Energy

In [ ]:
# Define unique colors for each plot
colors = [:blue, :red, :green, :orange, :purple, :brown]

# Container for each individual plot
plot_list = []

for i in N
    nse_values = Float64[]
    hours = Int[]
    # Loop through each hour to extract SOC values for storage unit 's_id'
    for h in H
        push!(nse_values, value(NSE[h, i]))  # Store SOC values
        push!(hours, h)                     # Store the corresponding hour
    end
    # Create the plot for this series with its own color and title
    p = plot(hours, nse_values, label="NSE$(i)", linewidth=2,
             linestyle=:solid, color=colors[i],
             xlims=(2538,2586),
             xlabel="Hour", ylabel="NSE (MWh)",
             title="NSE Zone $(i)")
    push!(plot_list, p)
end

# Arrange the 6 plots in a grid layout (3 rows x 2 columns)
plot(plot_list..., layout=(3, 2), size=(1300,900), subplot_padding=10)

**Last Day**

In [ ]:
#Demand
plot(load.Load_MW_z1+load.Load_MW_z2+load.Load_MW_z3+load.Load_MW_z4+load.Load_MW_z5+load.Load_MW_z6, xlims=(8688,8736) ,label="")
title!("Highest demand day")
xaxis!("Hour")
yaxis!("MWh")

In [ ]:

#Generators
# Generators
plot_list = []

# Loop through each zone and create its corresponding plot
for z in unique(df_gen_values.Zone)
    # Filter data for the current zone
    df_zone = filter(row -> row.Zone == z, df_gen_values)

    # Skip if no generators in this zone
    if nrow(df_zone) == 0
        println("No generators in zone $z")
        continue
    end

    # Create a new plot for this zone
    p = plot(
        title = "Generation Over Time (Zone $z)",
        xlabel = "Hour", ylabel = "Generation (MWh)",
        xlims = (8688,8736)
    )

    # Plot each generator in this zone
    for g in unique(df_zone.generators)
        df_g = df_zone[df_zone.generators .== g, :]
        resource_name = df_g.Resource[1]
        plot!(p, df_g.hour, df_g.generation, label = "$resource_name", linewidth = 2)
        resource_name = df_g.Resource[1]
    end

    # Add the current plot to the list
    push!(plot_list, p)
end

# Combine all the plots into a subplot grid.
# Adjust layout, overall figure size, and spacing as needed.
final_plot = plot(plot_list..., layout = (3, 2), size = (1200, 800), subplot_padding = 10)
display(final_plot)

In [ ]:
#Nodal
#Nodal prices
# Plot nodal prices for Node 1, Node 2, and Node 3
p12 = plot(H, nodal_prices_vector1, xlabel="Hour", ylabel="Nodal Price (USD/MWh)",
           title="Nodal Prices Over Time", label="Node 1", lw=2,
           xlims=(8688,8736), ylims=(0, 400),legend=:topright)

plot!(H, nodal_prices_vector2, label="Node 2", lw=2)
plot!(H, nodal_prices_vector3, label="Node 3", lw=2)
plot!(H, nodal_prices_vector4, label="Node 4", lw=2)
plot!(H, nodal_prices_vector5, label="Node 5", lw=2)
plot!(H, nodal_prices_vector6, label="Node 6", lw=2)


display(p12)

**First Day**

In [ ]:
#Demand
#Demand
plot(load.Load_MW_z1+load.Load_MW_z2+load.Load_MW_z3+load.Load_MW_z4+load.Load_MW_z5+load.Load_MW_z6, xlims=(2270,2280) ,label="")
title!("Highest demand day")
xaxis!("Hour")
yaxis!("MWh")

In [ ]:
#Generators
# Generators
plot_list = []

# Loop through each zone and create its corresponding plot
for z in unique(df_gen_values.Zone)
    # Filter data for the current zone
    df_zone = filter(row -> row.Zone == z, df_gen_values)

    # Skip if no generators in this zone
    if nrow(df_zone) == 0
        println("No generators in zone $z")
        continue
    end

    # Create a new plot for this zone
    p = plot(
        title = "Generation Over Time (Zone $z)",
        xlabel = "Hour", ylabel = "Generation (MWh)",
        xlims = (2270,2280)
    )

    # Plot each generator in this zone
    for g in unique(df_zone.generators)
        df_g = df_zone[df_zone.generators .== g, :]
        resource_name = df_g.Resource[1]
        plot!(p, df_g.hour, df_g.generation, label = "$resource_name", linewidth = 2)
        resource_name = df_g.Resource[1]
    end

    # Add the current plot to the list
    push!(plot_list, p)
end

# Combine all the plots into a subplot grid.
# Adjust layout, overall figure size, and spacing as needed.
final_plot = plot(plot_list..., layout = (3, 2), size = (1200, 800), subplot_padding = 10)
display(final_plot)

In [ ]:
#Nodal
#Nodal prices
# Plot nodal prices for Node 1, Node 2, and Node 3
p12 = plot(H, nodal_prices_vector1, xlabel="Hour", ylabel="Nodal Price (USD/MWh)",
           title="Nodal Prices Over Time", label="Node 1", lw=2,
           xlims=(2270,2280),legend=:topright)

plot!(H, nodal_prices_vector2, label="Node 2", lw=2)
plot!(H, nodal_prices_vector3, label="Node 3", lw=2)
plot!(H, nodal_prices_vector4, label="Node 4", lw=2)
plot!(H, nodal_prices_vector5, label="Node 5", lw=2)
plot!(H, nodal_prices_vector6, label="Node 6", lw=2)


display(p12)
savefig(p12, "price_flact_nonflex.pdf")

**NSE**

In [ ]:
plot_list = []
a=0
for i in N
    nse_values = Float64[]
    hours = Int[]
    for h in H
        push!(nse_values, value(NSE[h, i]))  
        push!(hours, h)                     
        a=a+value(NSE[h, i])
    end
end

# Arrange the 6 plots in a grid layout (3 rows x 2 columns)
print(a)

**Value of Flexibility for EV owners**

In [ ]:
sum(nodal_prices_vector1.*flex_avail.ev_load_shifting_z1+nodal_prices_vector2.*flex_avail.ev_load_shifting_z2+nodal_prices_vector3.*flex_avail.ev_load_shifting_z3+nodal_prices_vector4.*flex_avail.ev_load_shifting_z4+nodal_prices_vector5.*flex_avail.ev_load_shifting_z5+nodal_prices_vector6.*flex_avail.ev_load_shifting_z6)

In [ ]:
# Function to print the CDF values
function print_cdf(node_prices_dict, node_label)
    # Convert dictionary values to an array
    prices = collect(values(node_prices_dict))
    # Compute empirical CDF
    F = ecdf(prices)
    # Sort the data so we can print them in ascending order
    sorted_values = sort(prices)
    cdf_values = [F(x) for x in sorted_values]

    println(" CDF for $node_label")
    println("Price\tCDF")
    for (val, cdf_val) in zip(sorted_values, cdf_values)
        println("$val\t$cdf_val")
    end
    println()  # Blank line for readability
end



# Plot the CDF for each node on the same figure
p = plot(title = "CDF for Nodal Prices by Node",
         xlabel = "Nodal Price \$/MWh", ylabel = "Cumulative Probability")

# Collect all node dictionaries in an array for convenience
nodes = [nodal_prices_node1, nodal_prices_node2, nodal_prices_node3,
         nodal_prices_node4, nodal_prices_node5, nodal_prices_node6]

for (i, node_dict) in enumerate(nodes)
    # Extract and sort the nodal prices
    prices = sort(collect(values(node_dict)))
    # Compute CDF
    F = ecdf(prices)
    cdf_vals = [F(x) for x in prices]
    # Step plot is typical for an empirical CDF
    plot!(prices, cdf_vals, seriestype = :step,  label = "Node $i")
end

# Display the combined plot
display(p)
# Save it as a PDF
savefig(p, "cdf_nonflex.pdf")

In [ ]:
# Function to print the CDF values
function print_cdf(node_prices_dict, node_label)
    # Convert dictionary values to an array
    prices = collect(values(node_prices_dict))
    # Compute empirical CDF
    F = ecdf(prices)
    # Sort the data so we can print them in ascending order
    sorted_values = sort(prices)
    cdf_values = [F(x) for x in sorted_values]

    println("Empirical CDF for $node_label")
    println("Price\tCDF")
    for (val, cdf_val) in zip(sorted_values, cdf_values)
        println("$val\t$cdf_val")
    end
    println()  # Blank line for readability
end



# Plot the CDF for each node on the same figure
p = plot(title = "CDF for Nodal Prices by Node",
         xlabel = "Nodal Price \$/MWh", ylabel = "Cumulative Probability")

# Collect all node dictionaries in an array for convenience
nodes = [nodal_prices_node1, nodal_prices_node2, nodal_prices_node3,
         nodal_prices_node4, nodal_prices_node5, nodal_prices_node6]

for (i, node_dict) in enumerate(nodes)
    # Extract and sort the nodal prices
    prices = sort(collect(values(node_dict)))
    # Compute CDF
    F = ecdf(prices)
    cdf_vals = [F(x) for x in prices]
    # Step plot is typical for an empirical CDF
    plot!(prices, cdf_vals, seriestype = :step, xlims=(0,310), label = "Node $i")
end

# Display the combined plot
display(p)
# Save it as a PDF
savefig(p,"cdf_noflex2.pdf")
